In [12]:
import os
from google.colab import drive
from os.path import join as jn
import shutil

# 구글 드라이브 접근
ROOT = "/content/drive"
try:
  drive.mount(ROOT, force_remount=True)
except:
  drive.mount(ROOT)

# 본인 경로에 맞게 수정하면 됨.
PATH = jn(ROOT, "MyDrive/Task/plastic-segmentation/Data")
# PATH = jn(ROOT, "MyDrive/Task/plastic-segmentation/Sample_data")

# 구글 드라이브 경로에서 '/content/sample_data/data"로 복사 시
# shutil.copytree(PATH +"/train", "./sample_data/data/train")
# shutil.copytree(PATH +"/test", "./sample_data/data/test")

Mounted at /content/drive


# init

In [13]:
import os
import json
from tqdm import tqdm
from glob import glob

import numpy as np
# import pandas as pd
import cv2 as cv
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

import torch
import torch.optim as optim
from torch  import nn, Tensor

BASE = os.getcwd()
PATH = PATH # gdrive
# PATH = f"{BASE}/sample_data/data"   # colab
# PATH = f"{BASE}/assets/data"        # github

In [15]:
# version 1 : @qkrwjdduf159

## train
trn_path = f"{PATH}/train/annotation"
file_list = os.listdir(trn_path)

train_files = []
for file in tqdm(file_list):
    dir = trn_path + '/' + file
    json_list = glob(dir + '/' + '*.json')
    train_files.append(json_list)

train_json_list = []
for files in tqdm(train_files):
    for json_file in tqdm(files):
        train_json_list.append(json_file)

## test
tst_PATH = f"{PATH}/test/annotations"
file_list = os.listdir(tst_PATH)

test_files = []
for file in tqdm(file_list):
    dir = tst_PATH + '/' + file
    json_list = glob(dir + '/' + '*.json')
    test_files.append(json_list)

test_json_list = []
for files in tqdm(test_files):
    for json_file in tqdm(files):
        test_json_list.append(json_file)

100%|██████████| 1000/1000 [00:00<00:00, 1985939.39it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1808669.25it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1076290.48it/s]

100%|██████████| 100/100 [00:00<00:00, 594094.05it/s]

100%|██████████| 100/100 [00:00<00:00, 884874.26it/s]

100%|██████████| 100/100 [00:00<00:00, 651289.44it/s]

100%|██████████| 4/4 [00:00<00:00, 189.45it/s]


In [14]:
# version 2 @AshbeeKim
trn = os.path.join(PATH, "train")
tst = os.path.join(PATH, "test")

baseDF = {"kind" : [], "label" : [], "metainfo_id" : [], "feature" : [], "image_path" : [], "annot_path": []}
trn_num, tst_num = 0, 0
for fpath in tqdm([trn, tst]):
    kind = os.path.basename(fpath)
    BDIR = sorted(os.listdir(fpath))    # image, annotation(s)
    for bdir in BDIR:
        for dirs in sorted(os.listdir(jn(fpath, bdir))):
            dpath = jn(fpath, bdir, dirs)
            if bdir.lower()=='image':
                paths = sorted(glob(dpath + "/*.jpg"))
                baseDF['image_path'].extend(paths)
                kinds = [kind for cnt in range(len(paths))]
                baseDF['kind'].extend(kinds)
                if kind=='train' : trn_num += len(paths)
                elif kind=='test' : tst_num += len(paths)
                fnames = [os.path.basename(fname) for fname in paths]
                labels = list(map(lambda x: (x.split('_')[0]), fnames))
                baseDF['label'].extend(labels)
                metaIds = list(map(lambda x: int(x.split('_')[1]), fnames))
                baseDF['metainfo_id'].extend(metaIds)
                feats = list(map(lambda x: int(x.split('_')[-1][:-4]), fnames))
                baseDF['feature'].extend(feats)
            else:
                paths = sorted(glob(dpath + "/*.json"))
                baseDF['annot_path'].extend(paths)

100%|██████████| 2/2 [00:15<00:00,  7.60s/it]


In [45]:
# cotributed by @qkrwjdduf156, @AshbeeKim
import os
import numpy as np
import torch
from PIL import Image

# 추가 수정 중
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, base_info, transforms = None, mode = 'train', train_size = None, test_size = None):
        self.mode = mode
        self.infos = base_info
        self.transforms = transforms
        self.train_size = train_size
        self.test_size = test_size
        
    def __getitem__(self, idx):
        if self.mode == 'train':
            json_path = (self.infos['annot_path'])[:self.train_size][idx]

            with open(json_path, 'r') as f:
                annot = json.loads(f.read())

            image_path = (self.infos['image_path'])[:self.train_size][idx]
            image = Image.open(image_path).convert('RGB')

            boxes = []
            segmentations = []
            labels = []
            for i in range(len(annot['annotations'])):

                segmentation = annot['annotations'][i]['segmentation'][0]
                bbox = annot['annotations'][i]['bbox']
                label = torch.as_tensor(annot['annotations'][i]['category_id'], dtype = torch.int64)
                xmin, ymin, width, height = bbox[0],bbox[1],bbox[2],bbox[3]
                xmin, ymin, xmax, ymax = xmin, ymin, xmin + width, ymin + height
                
                boxes.append([xmin, ymin, xmax, ymax])
                segmentations.append([segmentation])
                labels.append(label)

            target = {}
            target['boxes'] = boxes
            target['labels'] = labels
            target['segmentation'] = segmentations
            target['image_id'] = image_path.split('/')[-1]

            if self.transforms is not None:
                image, target = self.transforms(image, target)

            return image, target

        elif self.model == 'test':
            st_num = self.train_size
            ed_num = self.train_size + self.test_size

            json_path = ((self.infos['annot_path'])[st_num:ed_num])[idx]
            with open(json_path, 'r') as f:
                annot = json.loads(f.read())

            image_path = ((self.infos['image_path'])[st_num:ed_num])[idx]
            image = Image.open(image_path).convert('RGB')
            
            target = {}
            target['image_id'] = image_path.split('/')[-1]
            return image, target
        
    def __len__(self):
        if self.mode == 'train':
            return self.train_size
        elif self.mode == 'test':
            return self.test_size

In [46]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

train_dataset = CustomDataset(baseDF, mode = 'train', train_size=trn_num, test_size=tst_num)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

num_classes = 5 # 4 class + background
in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [47]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

model = FasterRCNN(backbone,
                   num_classes=5,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [48]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256

    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

*** 현재 수정 중 ***

In [53]:
from torchvision import utils
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = CustomDataset(baseDF, mode = 'train', train_size=trn_num, test_size=tst_num)
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=5, shuffle=True, num_workers=4,
 collate_fn=utils.collate_fn)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

AttributeError: ignored

In [50]:
from torchvision import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


# transform = T.Compose([
#         T.Resize(256),
#         T.RandomCrop(224),
#         T.RandomHorizontalFlip(),
#         T.ToTensor(),
#         T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

In [44]:
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

import numpy as np
from PIL import Image
from torchvision import transforms
# print(type(train_dataset))
input_image = train_dataset.__getitem__(23)[0]

input_image = np.squeeze(input_image.numpy())
# print(type(input_image[0]))
m, s = np.mean(input_image[0], axis=(0, 1)), np.std(input_image[0], axis=(0, 1))
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=m, std=s),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model = model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

print(torch.round(output[0]))

Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


AttributeError: ignored

In [39]:
input_image

(<PIL.Image.Image image mode=RGB size=2048x2048 at 0x7F003D7937D0>,
 {'boxes': [[1134, 765, 1322, 882],
   [1359, 1086, 1548, 1394],
   [768, 1044, 925, 1265]],
  'image_id': 'PE_074_121.jpg',
  'labels': [tensor(4), tensor(4), tensor(4)],
  'segmentation': [[[1167,
     765,
     1160,
     766,
     1153,
     766,
     1146,
     767,
     1139,
     770,
     1135,
     777,
     1135,
     784,
     1135,
     791,
     1135,
     798,
     1135,
     805,
     1135,
     812,
     1134,
     819,
     1134,
     826,
     1134,
     833,
     1134,
     840,
     1134,
     847,
     1134,
     854,
     1134,
     861,
     1135,
     868,
     1142,
     873,
     1149,
     874,
     1156,
     875,
     1163,
     876,
     1170,
     877,
     1177,
     877,
     1184,
     878,
     1191,
     878,
     1198,
     878,
     1205,
     879,
     1212,
     879,
     1219,
     880,
     1226,
     881,
     1233,
     881,
     1240,
     881,
     1247,
     882,
     1254